# Basket option implementation based on normal model

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
from option_models import basket
from option_models import bsm
from option_models import normal

In [34]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)

[1 0 0 0]


In [35]:
# Compare the price to normal model formula

norm1 = normal.NormalModel(vol=40)
price_norm = norm1.price(strike=120, spot=100, texp=texp, cp=1)
print(price_basket, price_norm)

26.763909734339606 26.570845957870503


In [36]:
# A trivial test case 2
# all assets almost perfectly correlated:
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 5
strike = 120

print( cor_m )

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_norm)

[[1.     0.9999 0.9999 0.9999]
 [0.9999 1.     0.9999 0.9999]
 [0.9999 0.9999 1.     0.9999]
 [0.9999 0.9999 0.9999 1.    ]]
26.761395615904505 26.570845957870503


In [37]:
# A full test set for basket option with exact price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 100
price_exact = 28.0073695

In [24]:
cor_m

[40. 40. 40. 40.]


In [38]:
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_exact)

28.168716545630815 28.0073695


# [To Do] Basket option implementation based on BSM model
## Write the similar test for BSM

In [41]:
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)
print(price_basket)

27.663184979854396


In [47]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)

[1 0 0 0]


In [49]:
bsm1 = bsm.BsmModel(vol=0.4)
price_bsm = bsm1.price(strike=120, spot=100, texp=texp, cp=1)
print(price_basket, price_bsm)

28.751631082489702 28.713486748445934


# Spread option implementation based on normal model

In [11]:
# A full test set for spread option

spot = np.array([100, 96])
vol = np.array([0.2, 0.1])
weights = np.array([1, -1])
divr = np.array([1, 1])*0.05
intr = 0.1
cor_m = np.array([[1, 0.5], [0.5, 1]])
texp = 1
strike = 0
price_exact = 8.5132252

In [12]:
# MC price based on normal model
# make sure that the prices are similar

np.random.seed(123456)
price_spread = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=False)
print(price_spread, price_exact)

8.348852807225292 8.5132252


# Spread option implementation based on BSM model

In [13]:
# Once the implementation is finished the BSM model price should also work
price_spread = basket.basket_price_mc(
    strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=True)

In [14]:
# You also test Kirk's approximation
price_kirk = basket.spread_price_kirk(strike, spot, vol, texp, 0.5, intr, divr)
print(price_kirk, price_spread)

8.5132252295455 0.0


# [To Do] Complete the implementation of basket_price_norm_analytic
# Compare the MC stdev of BSM basket prices from with and without CV

In [50]:
# The basket option example from above
spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [51]:
### Make sure that the analytic normal price is correctly implemented
basket.basket_price_norm_analytic(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr)

26.570845957870503

In [70]:
# Run below about 100 times and get the mean and stdev

### Returns 2 prices, without CV and with CV 
price_baskets = []
for i in range(100):
    price_basket = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)
    price_baskets.append(price_basket)

price_baskets = np.array(price_baskets)
print(price_baskets.shape)

(100, 2)


In [71]:
# prices without CV
print('prices without CV')
print("mean:{mean}, std:{std}".format(mean = price_baskets[:,0].mean(), std = price_baskets[:,0].std()))

prices without CV
mean:28.72015229145853, std:0.8989079055046135


In [63]:
# prices with CV
print('prices with CV')
print("mean:{mean}, std:{std}".format(mean = price_baskets[:,1].mean(), std = price_baskets[:,1].std()))

prices with CV
mean:28.679103896088883, std:0.6010211798422068
